In [8]:
import pytc
import plotly
import ipywidgets as widgets

from IPython.display import display
from IPython.html.widgets import interactive
from plotly.widgets import GraphWidget

In [13]:
def _upload():

    _upload_widget = fileupload.FileUploadWidget()

    def _cb(change):
        decoded = io.StringIO(change['owner'].data.decode('utf-8'))
        filename = change['owner'].filename
        print('Uploaded `{}` ({:.2f} kB)'.format(
            filename, len(decoded.read()) / 2 **10))

    _upload_widget.observe(_cb, names='data')
    display(_upload_widget)

_upload()

experiments = []
initial_exp = 1

def add_exp(b):
    initial_exp += 1
        
exp_text = widgets.Text()
add_exp_b = widgets.Button(description = "Add an Experiment")
#add_exp_b.on_click(add_exp)
display(exp_text, add_exp_b)

Uploaded `setup.py` (0.65 kB)
